<a href="https://colab.research.google.com/github/20pa1a1268/basicml/blob/main/Copy_of_stress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score,  classification_report

In [ ]:
df = pd.read_csv('train.csv', encoding='latin-1')
df["FID"]

0           885-853-140.9727412-15.55450456
1           939-948-81.31774203-12.96443946
2           898-907-84.49723585-16.30527867
3           881-893-90.37053724-15.72046849
4           809-811-62.76624216-19.21381887
                        ...                
369284       721-721-36.3775595-9.609940972
369285      984-978-74.91843255-13.76227385
369286    1025-1024-95.30919959-23.05125375
369287       798-803-78.44989695-16.7366864
369288      814-815-67.69738701-20.64955147
Name: FID, Length: 369289, dtype: object

In [ ]:
df.fillna(value=1,inplace=True)
df.isnull().sum()

MEAN_RR        0
MEDIAN_RR      0
SDRR           0
RMSSD          0
condition      0
MEAN_RR.1      0
MEDIAN_RR.1    0
Column1        0
FID            0
dtype: int64

In [ ]:
def apply_results(condition):
            if (condition == 'no stress'):
                return 0  # No Stress
            elif (condition == 'interruption' or condition=="time pressure"):
                return 1  # Stress
            else:
                return 0
df['results'] = df['condition'].apply(apply_results)
x = df["FID"]
y=df["results"]
cv = CountVectorizer(lowercase=False, strip_accents='unicode', ngram_range=(1, 1))
x = cv.fit_transform(df["FID"])

In [ ]:
models = []
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
X_train.shape, X_test.shape, y_train.shape

((295431, 736807), (73858, 736807), (295431,))

In [ ]:
print("Naive Bayes")
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(X_train, y_train)
predict_nb = NB.predict(X_test)
naivebayes = accuracy_score(y_test, predict_nb) * 100
print(naivebayes)
print(confusion_matrix(y_test, predict_nb))
print(classification_report(y_test, predict_nb))
models.append(('naive_bayes', NB))


Naive Bayes
74.00552411384007
[[31330  8735]
 [10464 23329]]
              precision    recall  f1-score   support

           0       0.75      0.78      0.77     40065
           1       0.73      0.69      0.71     33793

    accuracy                           0.74     73858
   macro avg       0.74      0.74      0.74     73858
weighted avg       0.74      0.74      0.74     73858



In [ ]:
print("SVM")
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(X_train, y_train)
predict_svm = lin_clf.predict(X_test)
svm_acc = accuracy_score(y_test, predict_svm) * 100
print(svm_acc)
print("CLASSIFICATION REPORT")
print(classification_report(y_test, predict_svm))
print("CONFUSION MATRIX")
print(confusion_matrix(y_test, predict_svm))
models.append(('svm', lin_clf))

SVM
77.24146334858784
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.77      0.82      0.80     40065
           1       0.77      0.72      0.74     33793

    accuracy                           0.77     73858
   macro avg       0.77      0.77      0.77     73858
weighted avg       0.77      0.77      0.77     73858

CONFUSION MATRIX
[[32869  7196]
 [ 9613 24180]]


In [ ]:
print("Logistic Regression")
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, y_train)
y_pred = reg.predict(X_test)
print("ACCURACY")
print(accuracy_score(y_test, y_pred) * 100)
print("CLASSIFICATION REPORT")
print(classification_report(y_test, y_pred))
print("CONFUSION MATRIX")
print(confusion_matrix(y_test, y_pred))
models.append(('logistic', reg))

Logistic Regression
ACCURACY
76.86506539575943
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.77      0.81      0.79     40065
           1       0.76      0.72      0.74     33793

    accuracy                           0.77     73858
   macro avg       0.77      0.76      0.77     73858
weighted avg       0.77      0.77      0.77     73858

CONFUSION MATRIX
[[32498  7567]
 [ 9520 24273]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print("Decision Tree Classifier")
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtcpredict = dtc.predict(X_test)
print("ACCURACY")
print(accuracy_score(y_test, dtcpredict) * 100)
print("CLASSIFICATION REPORT")
print(classification_report(y_test, dtcpredict))
print("CONFUSION MATRIX")
print(confusion_matrix(y_test, dtcpredict))

Decision Tree Classifier
ACCURACY
97.77410707032413
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     40065
           1       0.98      0.98      0.98     33793

    accuracy                           0.98     73858
   macro avg       0.98      0.98      0.98     73858
weighted avg       0.98      0.98      0.98     73858

CONFUSION MATRIX
[[39236   829]
 [  815 32978]]


In [ ]:
print("Convolution Neural Network (CNN)")
from sklearn.neural_network import MLPClassifier
mlpc = MLPClassifier().fit(X_train, y_train)
y_pred = mlpc.predict(X_test)
print("ACCURACY")
print(accuracy_score(y_test, y_pred) * 100)
print("CLASSIFICATION REPORT")
print(classification_report(y_test, y_pred))
print("CONFUSION MATRIX")
print(confusion_matrix(y_test, y_pred))
models.append(('MLPClassifier', mlpc))




Convolution Neural Network (CNN)
